In [1]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import sys
from os.path import abspath
sys.path.insert(0, abspath('..'))

from os.path import join

from sklearn import metrics
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from spherecluster import SphericalKMeans, VonMisesFisherMixture

import numpy as np
from tabulate import tabulate

import logging

import torch


from torchSTC.data import load_data
from torchSTC.modules import STC
from torchSTC.metrics import SpacePlot, Evaluate
from torchSTC.utils.cluster import SphericalKmeans, SphericalKmeansPlus

plot = SpacePlot()
eval = Evaluate()

[nltk_data] Downloading package punkt to /home/godwin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /home/godwin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


>>>>> /home/godwin/Documents/academic/PPD/torchSTC/demos/tests_notebook
>>>>> data_loader.py cwd:  /home/godwin/Documents/academic/PPD/torchSTC/demos/tests_notebook


/home/godwin/Documents/academic/PPD/torchSTC/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

In [3]:
cur = abspath("")
dataset = 'Biomedical'
data_in_dir=join(cur, "../..", "datasets")
dataset_dir=join(data_in_dir, dataset)

# Word2Vec

In [3]:
checkpoint_dir=join(dataset_dir, 
                    "artefacts",
                    "STC-d48:500:2000:20-epoch30-datBiomedical-wdeWord2Vec-scaMinMax-tfeSIF-normNone-initKmeans")

checkpoint = "STC-datBiomedical-wdeWord2Vec-scaMinMax-tfeSIF-normNone-initKmeans.pth"
checkpoint_path = join(checkpoint_dir, checkpoint)
checkpoint_path

'/home/godwin/Documents/academic/PPD/torchSTC/demos/tests_notebook/../../datasets/Biomedical/artefacts/STC-d48:500:2000:20-epoch30-datBiomedical-wdeWord2Vec-scaMinMax-tfeSIF-normNone-initKmeans/STC-datBiomedical-wdeWord2Vec-scaMinMax-tfeSIF-normNone-initKmeans.pth'

In [4]:
x, y = load_data(dataset=dataset_dir, word_emb='Word2Vec', transform='SIF', scaler='MinMax', norm=None)
n_clusters = len(torch.unique(torch.tensor(y)))

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=0)

# conversion des données en tenseurs
X_train = torch.tensor(X_train, dtype=torch.float)
X_test = torch.tensor(X_test, dtype=torch.float)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

stc = STC(hidden_dims=[torch.Tensor(X_train).shape[-1], 500, 2000, 20], n_clusters=n_clusters)
stc.from_pretrained(checkpoint_path)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

### Embedding started...
Test bio w2v bis ==> ==> ==> 
Word2Vec words embedding loaded...
#### SIF embedding started...
SIF-Embedding 19999 documents with 48-dimensional word vectors...
SVD decomposition...
### SIF embedding completed...
### Embedding completed...
[embed_docs] XX shape:  (20000, 48)
MinMax scaling completed...
No normalization applied...


(torch.Size([18000, 48]),
 torch.Size([2000, 48]),
 torch.Size([18000]),
 torch.Size([2000]))

In [5]:
z2 = stc.autoencoder.encoder(X_train)

In [6]:
# comprehension list with 5 runs of kmeans, get average and std of metrics
avg_w2v_mmx_ikm2 = []
tmp = []
for i in range(5):
    kmeans = KMeans(n_clusters=n_clusters, n_init=100)
    kmeans.fit(z2.detach().numpy())
    y_km_pred = kmeans.labels_
    tmp.append(eval.allMetrics(y_train.detach().numpy(), y_km_pred))

avg_w2v_mmx_ikm2 = np.array(tmp)
np.round(avg_w2v_mmx_ikm2.mean(axis=0), 3) * 100, avg_w2v_mmx_ikm2.std(axis=0)

(array([45.7, 37.1, 26.6]),
 array([4.89897949e-05, 4.00000000e-05, 4.00000000e-05]))

## HuggingFace - SkM

In [4]:
checkpoint_dir=join(dataset_dir, 
                    "artefacts",
                    "STC-d384:500:2000:20-epoch30-datBiomedical-wdeHuggingFace-scaMinMax-tfeSIF-normNone-initSphericalKmeans"
                    )

checkpoint = "STC-datBiomedical-wdeHuggingFace-scaMinMax-tfeSIF-normNone-initSphericalKmeans.pth"
checkpoint_path = join(checkpoint_dir, checkpoint)
checkpoint_path

'/home/godwin/Documents/academic/PPD/torchSTC/demos/tests_notebook/../../datasets/Biomedical/artefacts/STC-d384:500:2000:20-epoch30-datBiomedical-wdeHuggingFace-scaMinMax-tfeSIF-normNone-initSphericalKmeans/STC-datBiomedical-wdeHuggingFace-scaMinMax-tfeSIF-normNone-initSphericalKmeans.pth'

In [5]:
x, y = load_data(dataset=dataset_dir, word_emb='HuggingFace', transform='SIF', scaler='MinMax', norm=None)
n_clusters = len(torch.unique(torch.tensor(y)))

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=0)

# conversion des données en tenseurs
X_train = torch.tensor(X_train, dtype=torch.float)
X_test = torch.tensor(X_test, dtype=torch.float)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

stc = STC(hidden_dims=[torch.Tensor(X_train).shape[-1], 500, 2000, 20], n_clusters=n_clusters)
stc.from_pretrained(checkpoint_path)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

MinMax scaling completed...


(torch.Size([18000, 384]),
 torch.Size([2000, 384]),
 torch.Size([18000]),
 torch.Size([2000]))

In [6]:
z3 = stc.autoencoder.encoder(X_train)

In [7]:
# comprehension list with 5 runs of kmeans, get average and std of metrics
avg_w2v_mmx_ikm3 = []
tmp = []
for i in range(5):
    kmeans = KMeans(n_clusters=n_clusters, n_init=100)
    kmeans.fit(z3.detach().numpy())
    y_km_pred = kmeans.labels_
    tmp.append(eval.allMetrics(y_train.detach().numpy(), y_km_pred))

avg_w2v_mmx_ikm3 = np.array(tmp)
np.round(avg_w2v_mmx_ikm3.mean(axis=0), 3) * 100, avg_w2v_mmx_ikm3.std(axis=0)

(array([45.4, 38.6, 26.4]),
 array([4.89897949e-05, 7.48331477e-05, 7.48331477e-05]))

In [8]:
# comprehension list with 5 runs of kmeans, get average and std of metrics
avg_w2v_mmx_iskm3 = []
tmp = []
for i in range(5):
    kmeans = SphericalKmeans(n_clusters, n_init=100)
    kmeans.fit(z3.detach().numpy())
    y_km_pred = kmeans.labels_
    tmp.append(eval.allMetrics(y_train.detach().numpy(), y_km_pred))

avg_w2v_mmx_iskm3 = np.array(tmp)
np.round(avg_w2v_mmx_iskm3.mean(axis=0), 3) * 100, avg_w2v_mmx_iskm3.std(axis=0)

iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 18
iteration: 19
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 18
iteration: 19
iteration: 0
iteration: 1
iterat

(array([43.1, 37.6, 25.6]), array([0.0090681 , 0.00471652, 0.0044445 ]))

In [9]:
from scipy.sparse import csr_matrix

# comprehension list with 5 runs of kmeans, get average and std of metrics
avg_w2v_mmx_iskmp3 = []
tmp = []
for i in range(5):
    skmeanspp = SphericalKmeansPlus(n_clusters=n_clusters)
    x = csr_matrix(z3.detach().numpy())
    skmeanspp.fit(x)
    y_km_pred = skmeanspp.labels_
    tmp.append(eval.allMetrics(y_train.detach().numpy(), y_km_pred))

avg_w2v_mmx_iskmp3 = np.array(tmp)
np.round(avg_w2v_mmx_iskmp3.mean(axis=0), 3) * 100, avg_w2v_mmx_iskmp3.std(axis=0)

(array([43. , 37.6, 25.5]), array([0.00770101, 0.00329582, 0.00779318]))

## HuggingFace - kM

In [10]:
checkpoint_dir=join(dataset_dir, 
                    "artefacts",
                    "STC-d384:500:2000:20-epoch30-datBiomedical-wdeHuggingFace-scaMinMax-tfeSIF-normNone-initKmeans"
                    )

checkpoint = "STC-datBiomedical-wdeHuggingFace-scaMinMax-tfeSIF-normNone-initKmeans.pth"
checkpoint_path = join(checkpoint_dir, checkpoint)
checkpoint_path

'/home/godwin/Documents/academic/PPD/torchSTC/demos/tests_notebook/../../datasets/Biomedical/artefacts/STC-d384:500:2000:20-epoch30-datBiomedical-wdeHuggingFace-scaMinMax-tfeSIF-normNone-initKmeans/STC-datBiomedical-wdeHuggingFace-scaMinMax-tfeSIF-normNone-initKmeans.pth'

In [11]:
x, y = load_data(dataset=dataset_dir, word_emb='HuggingFace', transform='SIF', scaler='MinMax', norm=None)
n_clusters = len(torch.unique(torch.tensor(y)))

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=0)

# conversion des données en tenseurs
X_train = torch.tensor(X_train, dtype=torch.float)
X_test = torch.tensor(X_test, dtype=torch.float)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

stc = STC(hidden_dims=[torch.Tensor(X_train).shape[-1], 500, 2000, 20], n_clusters=n_clusters)
stc.from_pretrained(checkpoint_path)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

MinMax scaling completed...


(torch.Size([18000, 384]),
 torch.Size([2000, 384]),
 torch.Size([18000]),
 torch.Size([2000]))

In [12]:
z4 = stc.autoencoder.encoder(X_train)

In [13]:
# comprehension list with 5 runs of kmeans, get average and std of metrics
avg_w2v_mmx_ikm4 = []
tmp = []
for i in range(5):
    kmeans = KMeans(n_clusters=n_clusters, n_init=100)
    kmeans.fit(z4.detach().numpy())
    y_km_pred = kmeans.labels_
    tmp.append(eval.allMetrics(y_train.detach().numpy(), y_km_pred))

avg_w2v_mmx_ikm4 = np.array(tmp)
np.round(avg_w2v_mmx_ikm4.mean(axis=0), 3) * 100, avg_w2v_mmx_ikm4.std(axis=0)

(array([51.3, 42.9, 30.3]),
 array([0.00000000e+00, 5.55111512e-17, 0.00000000e+00]))

In [14]:
# comprehension list with 5 runs of kmeans, get average and std of metrics
avg_w2v_mmx_iskm4 = []
tmp = []
for i in range(5):
    kmeans = SphericalKmeans(n_clusters, n_init=100)
    kmeans.fit(z4.detach().numpy())
    y_km_pred = kmeans.labels_
    tmp.append(eval.allMetrics(y_train.detach().numpy(), y_km_pred))

avg_w2v_mmx_iskm4 = np.array(tmp)
np.round(avg_w2v_mmx_iskm4.mean(axis=0), 3) * 100, avg_w2v_mmx_iskm4.std(axis=0)

iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 18
iteration: 19
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 18
iteration: 19
iteration: 0
itera

(array([46.4, 41.3, 28.1]), array([0.02314912, 0.00731125, 0.00870986]))

In [15]:
from scipy.sparse import csr_matrix

# comprehension list with 5 runs of kmeans, get average and std of metrics
avg_w2v_mmx_iskmp4 = []
tmp = []
for i in range(5):
    skmeanspp = SphericalKmeansPlus(n_clusters=n_clusters)
    x = csr_matrix(z4.detach().numpy())
    skmeanspp.fit(x)
    y_km_pred = skmeanspp.labels_
    tmp.append(eval.allMetrics(y_train.detach().numpy(), y_km_pred))

avg_w2v_mmx_iskmp4 = np.array(tmp)
np.round(avg_w2v_mmx_iskmp4.mean(axis=0), 3) * 100, avg_w2v_mmx_iskmp4.std(axis=0)

(array([48.3, 41.9, 28.5]), array([0.00746121, 0.00294917, 0.01263541]))